# Lab 6-1: Softmax Classification

Author: Seungjae Lee (이승재)

<div class="alert alert-warning">
    We use elemental PyTorch to implement linear regression here. However, in most actual applications, abstractions such as <code>nn.Module</code> or <code>nn.Linear</code> are used.
</div>

## Imports

In [239]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [240]:
# For reproducibility
torch.manual_seed(1)

## Softmax

Convert numbers to probabilities with softmax.

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [6]:
z = torch.FloatTensor([1, 2, 3])

PyTorch has a `softmax` function.

In [148]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([[0.2877, 0.4874, 0.3291, 0.2828, 0.2280],
        [0.3284, 0.2735, 0.2664, 0.4154, 0.3707],
        [0.3839, 0.2391, 0.4045, 0.3018, 0.4013]], grad_fn=<SoftmaxBackward>)


Since they are probabilities, they should add up to 1. Let's do a sanity check.

In [19]:
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss (Low-level)

For multi-class classification, we use the cross entropy loss.

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

where $\hat{y}$ is the predicted probability and $y$ is the correct probability (0 or 1).

In [224]:
z = torch.rand(3, 5, requires_grad=True)
print(z)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.0169, 0.3830, 0.8878, 0.7163, 0.2505],
        [0.0670, 0.4946, 0.6932, 0.0849, 0.0816],
        [0.4937, 0.3975, 0.8214, 0.0155, 0.4205]], requires_grad=True)
tensor([[0.1233, 0.1779, 0.2947, 0.2483, 0.1558],
        [0.1554, 0.2383, 0.2906, 0.1582, 0.1576],
        [0.2064, 0.1874, 0.2864, 0.1279, 0.1918]], grad_fn=<SoftmaxBackward>)


In [225]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([1, 4, 0])


In [226]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]])

In [227]:
print("y_one_hot\n", y_one_hot, "\n")
print("hypothesis\n", hypothesis, "\n")
print("-torch.log(hypothesis)\n", -torch.log(hypothesis), "\n")
print("y_one_hot * -torch.log(hypothesis)\n", y_one_hot * -torch.log(hypothesis), "\n")
print("(y_one_hot * -torch.log(hypothesis)).sum(dim=1)\n", 
      (y_one_hot * -torch.log(hypothesis)).sum(dim=1), "\n")
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

y_one_hot
 tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]]) 

hypothesis
 tensor([[0.1233, 0.1779, 0.2947, 0.2483, 0.1558],
        [0.1554, 0.2383, 0.2906, 0.1582, 0.1576],
        [0.2064, 0.1874, 0.2864, 0.1279, 0.1918]], grad_fn=<SoftmaxBackward>) 

-torch.log(hypothesis)
 tensor([[2.0927, 1.7266, 1.2218, 1.3933, 1.8591],
        [1.8620, 1.4344, 1.2359, 1.8441, 1.8474],
        [1.5780, 1.6743, 1.2503, 2.0562, 1.6512]], grad_fn=<NegBackward>) 

y_one_hot * -torch.log(hypothesis)
 tensor([[0.0000, 1.7266, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.8474],
        [1.5780, 0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<MulBackward0>) 

(y_one_hot * -torch.log(hypothesis)).sum(dim=1)
 tensor([1.7266, 1.8474, 1.5780], grad_fn=<SumBackward2>) 

tensor(1.7173, grad_fn=<MeanBackward1>)


## Cross-entropy Loss with `torch.nn.functional`

PyTorch has `F.log_softmax()` function.

In [228]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-2.0927, -1.7266, -1.2218, -1.3933, -1.8591],
        [-1.8620, -1.4344, -1.2359, -1.8441, -1.8474],
        [-1.5780, -1.6743, -1.2503, -2.0562, -1.6512]], grad_fn=<LogBackward>)

In [229]:
# High level
F.log_softmax(z, dim=1)

tensor([[-2.0927, -1.7266, -1.2218, -1.3933, -1.8591],
        [-1.8620, -1.4344, -1.2359, -1.8441, -1.8474],
        [-1.5780, -1.6743, -1.2503, -2.0562, -1.6512]],
       grad_fn=<LogSoftmaxBackward>)

PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood.

In [232]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.7173, grad_fn=<MeanBackward1>)

In [233]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.7173, grad_fn=<NllLossBackward>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [234]:
F.cross_entropy(z, y)

tensor(1.7173, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entropy Loss

In [334]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [335]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


In [336]:
def checkAccuracy(W, b):
    print(W)
    print(b)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    print(hypothesis)
    print("----------------")
    res = torch.max(hypothesis, dim=1)[1]
    print(res)
    print(y_train)
    print("----------------")    
    match = 0
    for i in range(8):
        if res[i] == y_train[i]:
            match += 1
    print(match, "/", y_train.size()[0])
    print(match / y_train.size()[0] * 100, "% accuracy")

In [337]:
checkAccuracy(W, b)

tensor([[-1.6429, -0.8499,  2.4928],
        [ 0.7231, -2.2532,  1.5301],
        [ 0.2087,  0.8192, -1.0279],
        [-0.0671,  0.9278, -0.8607]], requires_grad=True)
tensor([-2.1531e-08], requires_grad=True)
tensor([[2.3658e-02, 6.7666e-04, 9.7566e-01],
        [1.7783e-02, 2.0217e-01, 7.8005e-01],
        [1.3411e-03, 2.4644e-01, 7.5222e-01],
        [1.9756e-04, 7.3565e-01, 2.6416e-01],
        [5.9065e-01, 3.5765e-06, 4.0935e-01],
        [2.0591e-02, 9.7929e-01, 1.1413e-04],
        [9.6044e-01, 5.6807e-04, 3.8996e-02],
        [9.8805e-01, 1.4835e-04, 1.1805e-02]], grad_fn=<SoftmaxBackward>)
----------------
tensor([2, 2, 2, 1, 0, 1, 0, 0])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
----------------
7 / 8
87.5 % accuracy


## Training with `F.cross_entropy`

In [348]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [341]:
checkAccuracy(W, b)

tensor([[-10.2105,  -1.1448,  11.3551],
        [ -2.8180,  -1.5091,   4.3270],
        [  8.7516,  -1.7761,  -6.9756],
        [ -2.7641,   4.5773,  -1.8133]], requires_grad=True)
tensor([-3.6930e-06], requires_grad=True)
tensor([[7.0047e-10, 3.4334e-06, 1.0000e+00],
        [6.1901e-03, 7.8503e-02, 9.1531e-01],
        [3.9055e-13, 1.0204e-01, 8.9796e-01],
        [3.5657e-10, 8.9235e-01, 1.0765e-01],
        [9.0537e-02, 9.0039e-01, 9.0690e-03],
        [4.3358e-02, 9.5664e-01, 3.4321e-18],
        [9.0007e-01, 9.9925e-02, 2.7208e-11],
        [9.8330e-01, 1.6703e-02, 1.4417e-14]], grad_fn=<SoftmaxBackward>)
----------------
tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
----------------
8 / 8
100.0 % accuracy


## High-level Implementation with `nn.Module`

In [349]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [350]:
model = SoftmaxClassifierModel()

Let's try another new dataset.

In [351]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818


In [358]:
hypothesis = model.forward(x_train)
print(hypothesis)
print("----------------")
res = torch.max(hypothesis, dim=1)[1]
print(res)
print(y_train)
print("----------------")    
match = 0
for i in range(8):
    if res[i] == y_train[i]:
        match += 1
print(match, "/", y_train.size()[0])
print(match / y_train.size()[0] * 100, "% accuracy")

tensor([[-4.2894, -0.4193,  4.5199],
        [-2.9419,  0.4789,  2.5343],
        [-7.0699,  3.5619,  4.7770],
        [-6.1064,  4.2210,  3.1405],
        [ 0.9614,  1.0687, -2.1034],
        [ 2.3790,  3.5190, -3.9502],
        [ 3.1379,  1.9785, -4.5445],
        [ 4.5466,  2.4102, -6.2876]], grad_fn=<AddmmBackward>)
----------------
tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
----------------
8 / 8
100.0 % accuracy
